In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.getcwd()
os.listdir('/content/drive/MyDrive/mathematics_dataset-v1.0/train-easy')

['algebra__linear_1d.txt',
 'algebra__linear_1d_composed.txt',
 'algebra__linear_2d.txt',
 'algebra__linear_2d_composed.txt',
 'algebra__polynomial_roots.txt',
 'algebra__polynomial_roots_composed.txt',
 'algebra__sequence_next_term.txt',
 'algebra__sequence_nth_term.txt',
 'arithmetic__add_or_sub.txt',
 'arithmetic__add_or_sub_in_base.txt',
 'arithmetic__add_sub_multiple.txt',
 'arithmetic__div.txt',
 'arithmetic__mixed.txt',
 'arithmetic__mul.txt',
 'arithmetic__mul_div_multiple.txt',
 'arithmetic__nearest_integer_root.txt',
 'arithmetic__simplify_surd.txt',
 'calculus__differentiate.txt',
 'calculus__differentiate_composed.txt',
 'comparison__closest.txt',
 'comparison__closest_composed.txt',
 'comparison__kth_biggest.txt',
 'comparison__kth_biggest_composed.txt',
 'comparison__pair.txt',
 'comparison__pair_composed.txt',
 'comparison__sort.txt',
 'comparison__sort_composed.txt',
 'measurement__conversion.txt',
 'measurement__time.txt',
 'numbers__base_conversion.txt',
 'numbers__di

In [13]:
import nltk
import re
from nltk.corpus import words
nltk.download('words')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [11]:
def check(x,vocabulary):
  if x in vocabulary:
    print(f"{x} presente in vocabulary.")
  else:
    print(f"{x} non è presente in vocabulary.")



In [12]:
class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.word_to_id = {word: idx/10000 for idx, word in enumerate(vocab)}

    def tokenize(self, text):
        tokens = re.findall(r'\d|\.|[-=+*/()?]|\S+', text)



        token_id = []
        unknown_tokens = []

        new_tokens = []
        for token in tokens:
          flag=0
          if '.' in token and token != '.':
            token=token=token.split('.')[0]
            flag=1
          if token in self.word_to_id:
            token_id.append(self.word_to_id[token])
          else:
                token_id.append(self.word_to_id['unknown'])
                unknown_tokens.append(token)
          if(flag==1):
            token_id.append(self.word_to_id['.'])


        if unknown_tokens:
            print("Parole sconosciute:", unknown_tokens)

        return token_id


math_vocab = ['0','1','2','3','4','5','6','7','8','9','.','+',',','-','*','/', '?','(',')']
english_words = words.words()
vocab = math_vocab+english_words
tokenizer = Tokenizer(vocab)

text = "(3*2?). "
token_ids = tokenizer.tokenize(text)
print("Token IDs:", token_ids)



Token IDs: [0.0017, 0.0003, 0.0014, 0.0002, 0.0016, 0.0018, 0.001]
